# How Well Did My Bot Trade?



For this analysis we only use a few great python3 libraries: pandas, plotly,requests, and datetime<br>
Lets import all of our libraries and sign into plotly

In [33]:
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import requests
from datetime import datetime, timedelta
plotly.tools.set_credentials_file(username='dannymoscoso05', api_key='OfRkZGSqKlcgLkBPG6rY')
plotly.tools.set_config_file(world_readable=True,)

The first step is load our data into pandas and fix the dates

In [34]:
df = pd.read_csv('trans01.csv')
df['Date'] = pd.to_datetime(df['Date'])

This dataset was taken from my Oanda account transaction history. All these trades were made with a python script I wrote to apply a simple technical analysis strategy, more about this trading strategy can be found here: 
https://tradingstrategyguides.com/parabolic-sar-moving-average-trade-strategy/.
The script was able to make trades based on this strategy, 
however it made a lot more losing trades than winning ones,
lets explore how well it performed.


In [35]:
df.head()

,Ticket,Date,Timezone,Transaction,Details,Instrument,Price,Units,Direction,Spread Cost,Stop Loss,Take Profit,Trailing Stop,Financing,Commission,P/L,Amount,Balance
0,6312,2018-08-29 17:25:11,EDT,ORDER_FILL,MARKET_ORDER_TRADE_CLOSE,USD/CAD,1.29076,650.0,Sell,0.1755,NaN,NaN,NaN,-0.0001,0.0,-0.2210,NaN,81.79
1,6313,2018-08-29 17:25:11,EDT,ORDER_CANCEL,LINKED_TRADE_CLOSED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6314,2018-08-29 17:25:11,EDT,ORDER_CANCEL,LINKED_TRADE_CLOSED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6315,2018-08-29 17:25:14,EDT,MARKET_ORDER,TRADE_CLOSE,USD/JPY,NaN,650.0,Sell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6316,2018-08-29 17:25:14,EDT,ORDER_FILL,MARKET_ORDER_TRADE_CLOSE,USD/JPY,111.67100,650.0,Sell,0.1240,NaN,NaN,NaN,0.0019,0.0,0.2854,NaN,82.08


Since there is a lot going on in this transaction history, I just want to filter out the trades had an account balance.

In [36]:
DateAndBalance = df.loc[df['Balance'] > 0, ['Date','Balance']]

This is what our new dataframe looks like:

In [31]:
DateAndBalance.head()

,Date,Balance
0,2018-08-29 17:25:11,81.79
4,2018-08-29 17:25:14,82.08
8,2018-08-29 17:25:45,82.08
12,2018-08-29 17:48:27,81.70
16,2018-08-30 02:56:16,81.70


In [37]:
trace0 = go.Scatter(
    x = DateAndBalance.Date.tolist(),
    y = DateAndBalance.Balance.tolist(),
    mode = 'lines+markers',
    marker={'color': 'purple'}, 
    name='1st Trace'
)

data = [trace0]
layout=go.Layout(title="Balance of Account", xaxis={'title':'Date Time'}, yaxis={'title':'Balance'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='Balance August29-30')

<plotly.tools.PlotlyDisplay object>


In [12]:
df.tail(5)

,Ticket,Date,Timezone,Transaction,Details,Instrument,Price,Units,Direction,Spread Cost,Stop Loss,Take Profit,Trailing Stop,Financing,Commission,P/L,Amount,Balance
86,6398,2018-08-30 14:08:07,EDT,ORDER_CANCEL,LINKED_TRADE_CLOSED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,6399,2018-08-30 14:11:29,EDT,MARKET_ORDER,CLIENT_ORDER,EUR/USD,NaN,375.0,Buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,6400,2018-08-30 14:11:29,EDT,ORDER_FILL,MARKET_ORDER,EUR/USD,1.16686,375.0,Buy,0.0316,NaN,NaN,NaN,0.0,0.0,0.0,NaN,81.36
89,6401,2018-08-30 14:11:30,EDT,TAKE_PROFIT_ORDER,CLIENT_ORDER,NaN,1.16801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,6402,2018-08-30 14:11:30,EDT,STOP_LOSS_ORDER,CLIENT_ORDER,NaN,1.16551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
Losses = df.loc[df['P/L'] < 0, ['Date','Instrument','P/L','Balance']]
Profits = df.loc[df['P/L'] > 0, ['Date','Instrument','P/L','Balance']]
print('Lost Trades: '+str(len(Losses)))
print('Winning Trades: '+str(len(Profits)))

Lost Trades: 8
Winning Trades: 7


In [14]:
Losses['Instrument'].value_counts()

EUR/USD    6
USD/CAD    2
Name: Instrument, dtype: int64

In [15]:
Profits['Instrument'].value_counts()

EUR/USD    6
USD/JPY    1
Name: Instrument, dtype: int64

In [16]:
pairprofits =  Profits['Instrument'].value_counts()
pairlosses =  Profits['Instrument'].value_counts()
pairs =  Profits['Instrument'].value_counts().index.tolist()

In [17]:
profitloss = []
for x in pairs:
    numoflosses = len(Losses.loc[Losses['Instrument'] == x,])
    numofwins = len(Profits.loc[Profits['Instrument'] == x,])
    profitloss.append([x,numoflosses,numofwins])
profitloss.sort(key=lambda x: -x[2])
profitloss

[['EUR/USD', 6, 6], ['USD/JPY', 0, 1]]

In [18]:
fromtimeg = Profits.Date.tolist()[0] - timedelta(hours=6)
fromtimeg = fromtimeg.isoformat().replace(':','%3A') 
tog = Profits.Date.tolist()[0] 
tog = tog.isoformat().replace(':','%3A') 

In [19]:
pair = 'USD_CAD'

req = requests.get('https://api-fxpractice.oanda.com/v3/instruments/'+pair+'/candles?price=M&granularity=M1&from='+fromtimeg+'&to='+tog,headers=headers)
get = req.json()
timestamp=[]

o=[]
h=[]
l=[]
c=[]

for x in get['candles']:
    timestamp.append(x['time'])
    o.append(float(x['mid']['o']))
    h.append(float(x['mid']['h']))
    l.append(float(x['mid']['l']))
    c.append(float(x['mid']['c']))

df = pd.DataFrame(
{"timestamp" : timestamp,
"open" : o,
"high" : h,
"low" : l,
"close" : c,
},)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [20]:
len(df)

347

In [21]:
trace0 = go.Scatter(
    x = df.timestamp.tolist(),
    y = df.close.tolist(),
    mode = 'lines+markers',
    marker={'color': 'blue'}, 
    name='1st Trace'

)

data = [trace0]
layout=go.Layout(title="Exploring Price", xaxis={'title':'Date Time'}, yaxis={'title':'Close Price'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='Close')

In [22]:
Losses

,Date,Instrument,P/L,Balance
0,2018-08-29 17:25:11,USD/CAD,-0.2210,81.79
12,2018-08-29 17:48:27,USD/CAD,-0.3770,81.70
25,2018-08-30 03:58:05,EUR/USD,-0.6687,81.59
37,2018-08-30 05:34:16,EUR/USD,-0.6256,81.57
49,2018-08-30 07:12:16,EUR/USD,-0.6593,81.47
67,2018-08-30 10:42:43,EUR/USD,-0.6229,82.07
73,2018-08-30 11:24:56,EUR/USD,-0.6624,81.41
79,2018-08-30 13:07:18,EUR/USD,-0.6181,80.79


In [23]:
fromtimeg = Losses.Date.tolist()[0] - timedelta(hours=6)
fromtimeg = fromtimeg.isoformat().replace(':','%3A') 
tog = Profits.Date.tolist()[0] 
tog = tog.isoformat().replace(':','%3A') 

pair = 'GBP_USD'

req = requests.get('https://api-fxpractice.oanda.com/v3/instruments/'+pair+'/candles?price=M&granularity=M1&from='+fromtimeg+'&to='+tog,headers=headers)
get = req.json()
timestamp=[]

o=[]
h=[]
l=[]
c=[]

for x in get['candles']:
    timestamp.append(x['time'])
    o.append(float(x['mid']['o']))
    h.append(float(x['mid']['h']))
    l.append(float(x['mid']['l']))
    c.append(float(x['mid']['c']))

df = pd.DataFrame(
{"timestamp" : timestamp,
"open" : o,
"high" : h,
"low" : l,
"close" : c,
},)
df['timestamp'] = pd.to_datetime(df['timestamp'])


trace0 = go.Scatter(
    x = df.timestamp.tolist(),
    y = df.close.tolist(),
    mode = 'lines+markers',
    marker={'color': 'blue'}, 
    name='1st Trace'

)

data = [trace0]
layout=go.Layout(title="Exploring Price", xaxis={'title':'Date Time'}, yaxis={'title':'Close Price'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='Close')